<a href="https://colab.research.google.com/github/jks5177/BigAI/blob/main/%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC_4_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 머신러닝을 이용한 감성분석

## 문서 행렬 Document Term Matrix

* 비정형데이터 머신러닝은 기본적으로 문서 행렬을 이용한다.
* 일반적으로 DTM이라고 부른다.

1. 모든 문서에서 나타나는 어휘를 모으고, 일정 빈도 이상이 되는 것들로 컬럼을 만든다.
2. 각 문서에서 컬럼에 있는 어휘가 있는지 파악하고, 그 빈도를 기록한다.
3. 빈도는 단순 빈도(Term Frequency) 또는 TF-IDF 값이 될 수 있다.
4. 각 문서가 어떠한 종류에 속하는지를 기록하여 훈련 데이터를 만든다.

## tf-idf

* TF는 단순히 현재 문서에 출현한 단어의 빈도
* TF-IDF는 단어비도 * 역문서 빈도
* 역문서 빈도(IDF)는 전체 문서의 수를 그 단어가 나오는 문서의 수로 나눈 것

$IDF(t) = \log({전체 문서수}/{t가 나오는 문서 수(DF)})$

* IDF는 특정 문서에서만 나타나는 단어에 높은 값을 준다.
* 따라서 '영화, 재미있다' 등 모든 문서에서 많이 나타나는 단어들의 IDF값은 작다.
* 반면, '신기전, 라스베가스, 김지호' 등 특정 문서에서만 나타나는 단어들의 IDF 값은 크다.
* 여기에 일반적인 중요성을 나타내는 TF를 곱하게 되면 (TF*IDF)
* 그 문서에서 자주 나타난 단어(TF)가 몇몇 문서들에서만 자주 나타나는 단어(IDF)인 경우 높은 값을 갖게 된다.
* 즉, TD-IDF는 불용어의 가능성을 감안한 빈도이다.

# 사전 설정


## RHINO 설치

In [ ]:
# java jdk 설치
!apt-get update
!apt-get install g++ openjdk-8-jdk

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic

In [ ]:
#JPype 설치
!pip install JPype1

In [ ]:
# RHINO 설치
!pip install rhinoMorph

In [ ]:
# RHINO 시작
import rhinoMorph
rn = rhinoMorph.startRhino()

filepath:  /usr/local/lib/python3.7/dist-packages
classpath:  /usr/local/lib/python3.7/dist-packages/rhinoMorph/lib/rhino.jar
RHINO started!


## 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
!cp -r /content/gdrive/MyDrive/BigLeaderAI/자연어처리 /content/

In [ ]:
%cd /content/gdrive/MyDrive/BigLeaderAI/자연어처리/pytest/

/content/gdrive/MyDrive/BigLeaderAI/자연어처리/pytest


## Colab 폰트 설정

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina' # 폰트가 깨끗하게 보이도록 설정

!apt install fonts-nanum # 나눔 폰트 설치

import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumMyeongjo.ttf'
font = fm.FontProperties(fname=fontpath, size=9)

# 기본 글꼴 변경
import matplotlib as mpl
mpl.font_manager._rebuild()
mpl.pyplot.rc('font', family='NanumMyeongjo')

#런타임 > 런타임 다시 시작 수행 후 다시 진행

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 80 not upgraded.


# 데이터 로딩

In [ ]:
# 읽기 함수 정의
def read_data(filename, encoding='cp949', start_line=1) :
  with open(filename, 'r', encoding=encoding) as f :
    data = [line.split('\t') for line in f.read().splitlines()]
    data = data[start_line:]
  return data

#쓰기 함수도 정의
def write_data(data, filename, encoding='cp949') :
  with open(filename, 'w', encoding=encoding) as f :
    f.write(data)

In [ ]:
# 전체파일은 ratings.txt
# data = read_data('/content/자연어처리/pytest/ratings_small.txt', encoding='cp949') # 절대경로
data = read_data('ratings_morphed.txt', start_line=0) # 상대 경로

print(len(data))
print(len(data[0]))
print(data[0])

197441
3
['8132799', '디자인 배우 학생 외국 디자이너 일구 전통 통하 발전 문화 산업 부럽 사실 우리나라 그 어렵 시절 끝 열정 지키 노라노 같 전통 있 같 사람 꿈 꾸 이루 나가 있 감사', '1']


# 훈련데이터와 테스트데이터 분리(자동)

In [ ]:
data_text = [line[1] for line in data]
data_senti = [line[2] for line in data]

from sklearn.model_selection import train_test_split

train_data_text, test_data_text, train_data_senti, test_data_senti = train_test_split(data_text, data_senti, stratify=data_senti)

# Counter 클래스를 이용해 각 분류가 훈련데이터와 테스트데이터에 같은 비율로 들어갔는지 확인해 본다.
from collections import Counter
train_data_senti_freq = Counter(train_data_senti)
print('train_data_senti_freq:', train_data_senti_freq)

test_data_senti_freq = Counter(test_data_senti)
print('test_data_senti_freq:', test_data_senti_freq)

train_data_senti_freq: Counter({'1': 74086, '0': 73994})
test_data_senti_freq: Counter({'1': 24696, '0': 24665})


# 훈련데이터와 테스트데이터 분리(수동)

In [ ]:
import random
random.shuffle(data) # 랜덤하게 섞는다.

data_70 = int( len(data)*0.7) # 전체 데이터 크기의 70% 숫자를 찾는다.
train_data = data[:data_70] # 앞에서 70% 부분을 잘라 훈련데이터로 사용
test_data = data[data_70:] # 그 뒷부분을 테스트 데이터로

print('train data length : ', len(train_data))
print('test data lenght : ', len(test_data))

# 훈련데이터 요수 분리
train_data_text = [line[1] for line in train_data] # 훈련데이터 본문
train_data_senti = [line[2] for line in train_data] # 훈련데이터 긍부정 부분

# 테스트데이텀 요소
test_data_text = [line[1] for line in test_data] # 테스트데이터 본문
test_data_senti = [line[2] for line in test_data] # 테스트데이터 긍부정 부분

train data length :  138208
test data lenght :  59233


# 행렬 형태로 변환

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(min_df=5).fit(train_data_text) # 최소 문서 빈도 5이상의 단어만
X_train = vect.transform(train_data_text) # 행렬 생성
print("X_train : ", repr(X_train))

X_train :  <138208x11104 sparse matrix of type '<class 'numpy.int64'>'
	with 761495 stored elements in Compressed Sparse Row format>


* 138,079 행 x 11,142 열의 행렬 생성
* 그 중 765,535 셀에만 데이터 입력
* 0.05% 채워지고, 99.5%는 빈 공간

# 행렬 내용 관찰

In [ ]:
feature_names = vect.get_feature_names()
print("특성 개수:", len(feature_names))
print('처음 20개 특성 : \n', feature_names[:20])
print('3000~5000까지의 특성: \n', feature_names[3000:500000])

특성 개수: 11104
처음 20개 특성 : 
 ['10점', '1빠', 'ai', 'cgv', 'ebs', 'jtbc', 'kbs', 'la', 'mb', 'mbc', 'naver', 'new', 'ok', 'sbs', 'sns', 'tv', 'ㄴㄴ', 'ㄷㄷ', 'ㅂㄷㅂㄷ', 'ㅂㅅ']
3000~5000까지의 특성: 
 ['막대', '막론', '막막', '막말', '막무가내', '막바지', '막상', '막스', '막연', '막이', '막장', '막장일', '막지', '막판', '막히', '만감', '만그', '만끽', '만나', '만날', '만남', '만능', '만담', '만들', '만듦새', '만땅', '만만', '만발', '만보', '만사', '만세', '만약', '만인', '만점', '만족', '만족감', '만지', '만찬', '만하', '만해', '만행', '만화', '만화가', '만화영화', '만화책', '만회', '많이', '말기', '말길', '말년', '말론', '말리', '말미', '말세', '말씀', '말아먹', '말입', '말자', '말장난', '말종', '말초신경', '말코비치', '말콤', '말타', '말투', '말하', '말하기', '맘껏', '맘대로', '맘마미아', '맙소사', '맛깔', '맛나', '맛보', '맛보기', '맛없', '맛있', '맛집', '망가', '망가뜨리', '망가지', '망각', '망상', '망설이', '망설임', '망신', '망쳐놓', '망치', '망토', '망하', '맞먹', '맞물리', '맞서', '맞이', '맞추', '맞춤', '맡기', '매개체', '매국노', '매기', '매끄럽', '매너', '매너리즘', '매년', '매니아', '매달', '매달리', '매도', '매드', '매들린', '매력', '매력적', '매료', '매미', '매번', '매염방', '매우', '매운맛', '매일', '매일매일', '매장', '매주', '매직', '매진', '매체', '매춘부', '매치', '매튜', '매트', 

# 머신러닝 알고리즘 적용

In [ ]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

y_train = pd.Series(train_data_senti) # 리스트 형태의 종속변수가 될 수 있는 1차원 배열 생성

lr = LogisticRegression(solver = 'liblinear')
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

# 테스트 데이터 입력

In [ ]:
X_test = vect.transform(test_data_text)
y_test = pd.Series(test_data_senti)

print('테스트 데이터 점수 : ', lr.score(X_test, y_test))

테스트 데이터 점수 :  0.807387773707224


# 1개 데이터 예측

In [ ]:
# 형태소분석기 시작
import rhinoMorph
rn = rhinoMorph.startRhino()

new_input = '오늘은 정말 재미있는 하루구나!'
inputdata = []
morphed_input = rhinoMorph.onlyMorph_list(
    rn, new_input, pos=['NNG','NNP','VV','VA','XR','IC','MM','MAG','MAJ'])
morphed_input = ' '.join(morphed_input) # ['오늘', '정말', '재미있', '하루']를 한 개 문자열로 변환

inputdata.append(morphed_input) # 분석 결과를 리스트로 만들기
print('input data : ', inputdata) # ['오늘 정말 재미있 하루']

filepath:  /usr/local/lib/python3.7/dist-packages
classpath:  /usr/local/lib/python3.7/dist-packages/rhinoMorph/lib/rhino.jar
JVM is already started~
RHINO started!
input data :  ['오늘 정말 재미있 하루']


In [ ]:
X_input = vect.transform(inputdata) # 앞에서 만든 11445 컬럼의 행렬에 적용
result = lr.predict(X_input) # 0은 부정, 1은 긍정

if result == '0' :
  print('부정적인 글입니다.')
else :
  print('긍정적인 글입니다')

긍정적인 글입니다
